In [1]:
#install pyreadstat ==> to read spss file
#pip install pyreadstat

In this notebook, we will calculate consumption using Aguiar & Bills (2015) measure and calculate the Consmption Inequality index of WFH and non-WFH households across time.
Since the LCF survey is made up of thousands of respondents, we transform the data into a format that is suitable for a trend analysis.
In a nutshell, we will aggregate the incomes and expenses for each year, grouped by WFH and non-WFH, and combine them into one timeseries dataframe.

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import pyreadstat as pyrs
#set no limits on data display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('use_inf_as_na', True)
#getting the work directory
import os 
cwd = os.getcwd()
print(cwd)

C:\Users\OMEN\Anaconda3 Files


C:\Users\OMEN\AppData\Local\Temp\ipykernel_15832\3450391093.py:9: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


Import and clean the 'raw_per' table.
In the section below, we wil import the raw_per tables which contain the WFH indicators. This indicator is only available from the 2020 data.
We will also rename the WFH indicator to a standard naming convention, WfhCor.

In [3]:
#set list of required columns from the raw_per table.
req_cols_raw_per = ['case','Person','person','WfhCor','WfhCor2','wfhcor2', 'predicted_wfh']

In [4]:
#importing raw_per table from SAV to dataframe.
df_16_raw_per, meta_16_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2016_rawper_pseudo.sav", usecols=req_cols_raw_per)
df_17_raw_per, meta_17_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2017_rawper_pseudo.sav", usecols=req_cols_raw_per)
df_18_raw_per, meta_18_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2018_rawper_pseudo.sav", usecols=req_cols_raw_per)
df_19_raw_per, meta_19_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2019_rawper_pseudo.sav", usecols=req_cols_raw_per)
df_20_raw_per, meta_20_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2020_rawper_ukanon_final.sav", usecols=req_cols_raw_per)
df_21_raw_per, meta_21_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2021_rawper_ukanon_final.sav", usecols=req_cols_raw_per)
df_22_raw_per, meta_22_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2022_rawper_ukanon_final.sav", usecols=req_cols_raw_per)

In [5]:
### Delete old WfhCor indicator for prior 2020 data

df_16_raw_per = df_16_raw_per.drop('WfhCor', axis=1)
df_17_raw_per = df_17_raw_per.drop('WfhCor', axis=1)
df_18_raw_per = df_18_raw_per.drop('WfhCor', axis=1)
df_19_raw_per = df_19_raw_per.drop('WfhCor', axis=1)

In [6]:
#rename column WfhCor2 to WfhCor to match other tables
df_16_raw_per.rename(columns={"predicted_wfh": "WfhCor"}, inplace=True)
df_17_raw_per.rename(columns={"predicted_wfh": "WfhCor"}, inplace=True)
df_18_raw_per.rename(columns={"predicted_wfh": "WfhCor"}, inplace=True)
df_19_raw_per.rename(columns={"predicted_wfh": "WfhCor"}, inplace=True)
##### NO DF20 because its the base
df_21_raw_per.rename(columns={"WfhCor2": "WfhCor"}, inplace=True)
df_22_raw_per.rename(columns={"wfhcor2": "WfhCor"}, inplace=True)

In [7]:
#setting NaN values to 2 (which means no WFH).
df_20_raw_per['WfhCor'] = df_20_raw_per['WfhCor'].fillna(2)
df_21_raw_per['WfhCor'] = df_21_raw_per['WfhCor'].fillna(2)
df_22_raw_per['WfhCor'] = df_22_raw_per['WfhCor'].fillna(2)

#Setting the 0 values to 2 (which means no WFH)
df_16_raw_per['WfhCor'] = df_16_raw_per['WfhCor'].map({0:2, 1:1})
df_17_raw_per['WfhCor'] = df_17_raw_per['WfhCor'].map({0:2, 1:1})
df_18_raw_per['WfhCor'] = df_18_raw_per['WfhCor'].map({0:2, 1:1})
df_19_raw_per['WfhCor'] = df_19_raw_per['WfhCor'].map({0:2, 1:1})

In [8]:
#Group the dataframes by case ID
df_16_raw_per = df_16_raw_per.groupby(['case']).min()
df_17_raw_per = df_17_raw_per.groupby(['case']).min()
df_18_raw_per = df_18_raw_per.groupby(['case']).min()
df_19_raw_per = df_19_raw_per.groupby(['case']).min()
df_20_raw_per = df_20_raw_per.groupby(['case']).min()
df_21_raw_per = df_21_raw_per.groupby(['case']).min()
df_22_raw_per = df_22_raw_per.groupby(['case']).min()

In [9]:
#drop Person column
df_16_raw_per = df_16_raw_per.drop('Person', axis=1)
df_17_raw_per = df_17_raw_per.drop('Person', axis=1)
df_18_raw_per = df_18_raw_per.drop('Person', axis=1)
df_19_raw_per = df_19_raw_per.drop('Person', axis=1)
df_20_raw_per = df_20_raw_per.drop('Person', axis=1)
df_21_raw_per = df_21_raw_per.drop('Person', axis=1)
df_22_raw_per = df_22_raw_per.drop('person', axis=1)

In [10]:
#list of DFs
dataframes_raw = [df_16_raw_per,
                  df_17_raw_per,
                  df_18_raw_per,
                  df_19_raw_per,
                  df_20_raw_per,
                  df_21_raw_per,
                  df_22_raw_per]

#see how many in each df is earning salaried wages

for d in dataframes_raw:
    print(d['WfhCor'].value_counts())

WfhCor
2    3561
1    1480
Name: count, dtype: int64
WfhCor
2    3411
1    1996
Name: count, dtype: int64
WfhCor
2    3603
1    1870
Name: count, dtype: int64
WfhCor
2    3293
1    2145
Name: count, dtype: int64
WfhCor
2.0    3580
1.0    1820
Name: count, dtype: int64
WfhCor
2.0    3792
1.0    1840
Name: count, dtype: int64
WfhCor
2.0    3496
1.0     964
Name: count, dtype: int64


Import and clean dvhh table.
In the section below, we wil import the dv_hh tables which contain the derived household variables.
We will also rename the variable codes to human readable descriptions.

In [11]:
#set list of required columns from dvhh table
req_cols_dvhh = ['case','p425','p344p','p601t','p602t','p607t','p609t','p611t','p537t']

In [12]:
#importing dvhh table from SAV to dataframe
df_16, meta_16 = pyrs.read_sav("UK LCF Data Sets/2016_17_dvhh_ukanon.sav", usecols=req_cols_dvhh)
df_17, meta_17 = pyrs.read_sav("UK LCF Data Sets/2017_dvhh_ukanon_2017-18.sav", usecols=req_cols_dvhh)
df_18, meta_18 = pyrs.read_sav("UK LCF Data Sets/2018_dvhh_ukanon.sav", usecols=req_cols_dvhh)
df_19, meta_19 = pyrs.read_sav("UK LCF Data Sets/lcfs_2019_dvhh_ukanon.sav", usecols=req_cols_dvhh)
df_20, meta_20 = pyrs.read_sav("UK LCF Data Sets/lcfs_2020_dvhh_ukanon.sav", usecols=req_cols_dvhh)
df_21, meta_21 = pyrs.read_sav("UK LCF Data Sets/lcfs_2021_dvhh_ukanon.sav", usecols=req_cols_dvhh)
df_22, meta_22 = pyrs.read_sav("UK LCF Data Sets/lcfs_2022_dvhh_ukanon.sav", usecols=req_cols_dvhh)

Import data then join with raw_per table on case ID
In this section, we will join the data extracted from the raw_per table to the extracted dv_hh table using case ID.

In [13]:
#join raw_per table to dvhh table
df_16 = df_16.set_index('case').join(df_16_raw_per)
df_17 = df_17.set_index('case').join(df_17_raw_per)
df_18 = df_18.set_index('case').join(df_18_raw_per)
df_19 = df_19.set_index('case').join(df_19_raw_per)
df_20 = df_20.set_index('case').join(df_20_raw_per)
df_21 = df_21.set_index('case').join(df_21_raw_per)
df_22 = df_22.set_index('case').join(df_22_raw_per)

Calculating Aguiar and Bills' measeure of consumption; ratio of luxury/necessity spending. Aguiar and Bills categorised luxury spending as entertainment or recreation expense and necessity spending as food at home expense.

In [14]:
#list of DFs
dataframes = [df_16,df_17,df_18,df_19,df_20,df_21,df_22]

# calculate the derived consumption for each observation in each df

for d in dataframes:
    d['derived_consumption'] = (d['p609t']/d['p601t'])*100

Aggregate the income and expenses value and group by WFH indicator.
In this section, we separate the WFh and non-WFH respondents and calculate the 10th, and 90th percentile of each group's income and expenses.
The measures of expense uses the COICOP (Classification of individual consumption by purpose) used by the European Union, except for the measures of housing and utilities expenses which used the Office of National Statistics' (ONS) method of measurement. This is because ONS mesurement differentiated the housing expenditure e.g., rent as separate from utilities expenditure e.g., gas for central heating.

In [15]:
# Create conditional DataFrame column by np.where() function.

#create list and iterate to create wfh and non-wfh value columns
wfh_list = ['p344p-wfh','p601t-wfh','p602t-wfh','p607t-wfh','p609t-wfh','p611t-wfh','p537t-wfh','derived_consumption-wfh']
non_wfh_list = ['p344p-non_wfh','p601t-non_wfh','p602t-non_wfh','p607t-non_wfh','p609t-non_wfh','p611t-non_wfh','p537t-non_wfh','derived_consumption-non_wfh']
main_list = ['p344p','p601t','p602t','p607t','p609t','p611t','p537t','derived_consumption']
dataframes = [df_16,df_17,df_18,df_19,df_20,df_21,df_22]

for k in dataframes:
    for i, j, p in zip(wfh_list,main_list,non_wfh_list):
        k[i] = np.where(k['WfhCor']== 1, k[j], float('nan'))
        k[p] = np.where(k['WfhCor']== 2, k[j], float('nan'))

Export the cleaned data to csv

In [16]:
df_16.to_csv('LCFS_2016_Clean.csv', encoding='utf-8', index=True)
df_17.to_csv('LCFS_2017_Clean.csv', encoding='utf-8', index=True)
df_18.to_csv('LCFS_2018_Clean.csv', encoding='utf-8', index=True)
df_19.to_csv('LCFS_2019_Clean.csv', encoding='utf-8', index=True)
df_20.to_csv('LCFS_2020_Clean.csv', encoding='utf-8', index=True)
df_21.to_csv('LCFS_2021_Clean.csv', encoding='utf-8', index=True)
df_22.to_csv('LCFS_2022_Clean.csv', encoding='utf-8', index=True)

This is the data preprocessing step to compile the annual survey data into 1 dataframe for diff-in-diff analysis.
Columns required are: p425,p344p,derived_consumption,wfhcor. add column for year


Below is the diff-in-diff regression model.


In [45]:
# Read CSV file into DataFrame

req_cols = ['case', 'p425', 'p344p', 'derived_consumption', 'WfhCor']
df_16 = pd.read_csv('UK LCF Data Set Clean/LCFS_2016_Clean.csv', usecols = req_cols)
df_17 = pd.read_csv('UK LCF Data Set Clean/LCFS_2017_Clean.csv', usecols = req_cols)
df_18 = pd.read_csv('UK LCF Data Set Clean/LCFS_2018_Clean.csv', usecols = req_cols)
df_19 = pd.read_csv('UK LCF Data Set Clean/LCFS_2019_Clean.csv', usecols = req_cols)
df_20 = pd.read_csv('UK LCF Data Set Clean/LCFS_2020_Clean.csv', usecols = req_cols)
df_21 = pd.read_csv('UK LCF Data Set Clean/LCFS_2021_Clean.csv', usecols = req_cols)
df_22 = pd.read_csv('UK LCF Data Set Clean/LCFS_2022_Clean.csv', usecols = req_cols)

In [46]:
# Add the survey year indicator
df_16['year'] = 2016
df_17['year'] = 2017
df_18['year'] = 2018
df_19['year'] = 2019
df_20['year'] = 2020
df_21['year'] = 2021
df_22['year'] = 2022

In [47]:
# Concatenate the dfs into one

df_did = pd.concat([df_16, df_17, df_18, df_19, df_20, df_21, df_22])

In [51]:
# column names:

df_did.columns

Index(['case', 'p425', 'p344p', 'WfhCor', 'derived_consumption', 'year',
       'treatment'],
      dtype='object')

In [49]:
# remap wfhcor values

df_did['WfhCor'] = df_did['WfhCor'].map({2:0, 1:1})

In [50]:
# Add treatment indicator if year >=2020

df_did['treatment'] = np.where(df_did['year'] >=2020, 1,0 )

In [61]:
# Create dummy variables

df_did_clean = pd.get_dummies(df_did, prefix=['year','WfhCor', 'treatment'], prefix_sep='_', columns=['year','WfhCor','treatment'], drop_first=False)

In [62]:
df_did_clean.tail()

,case,p425,p344p,derived_consumption,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,WfhCor_0,WfhCor_1,treatment_0,treatment_1
4455,4456.0,1.0,1251.11,317.039883,False,False,False,False,False,False,True,True,False,False,True
4456,4457.0,2.0,409.63,13.431575,False,False,False,False,False,False,True,True,False,False,True
4457,4458.0,1.0,941.84,79.937362,False,False,False,False,False,False,True,False,True,False,True
4458,4459.0,2.0,1451.93,31.273255,False,False,False,False,False,False,True,True,False,False,True
4459,4460.0,1.0,1827.07,2.823008,False,False,False,False,False,False,True,True,False,False,True


In [63]:
# Drop the WfhCor_0 and treatment_0 columns
df_did_clean.drop('WfhCor_0', axis=1, inplace=True)
df_did_clean.drop('treatment_0', axis=1, inplace=True)

In [64]:
df_did_clean.head()

,case,p425,p344p,derived_consumption,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,WfhCor_1,treatment_1
0,1.0,1.0,844.864872,230.095704,True,False,False,False,False,False,False,False,False
1,2.0,1.0,873.197500,27.883846,True,False,False,False,False,False,False,True,False
2,3.0,5.0,333.914000,346.133146,True,False,False,False,False,False,False,False,False
3,4.0,5.0,1171.089500,145.377766,True,False,False,False,False,False,False,False,False
4,5.0,5.0,510.781500,27.224436,True,False,False,False,False,False,False,False,False


In [65]:
df_did_clean['did_effect'] = df_did_clean['WfhCor_1'] * df_did_clean['treatment_1']

In [67]:
df_did_clean.tail()

,case,p425,p344p,derived_consumption,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,WfhCor_1,treatment_1,did_effect
4455,4456.0,1.0,1251.11,317.039883,False,False,False,False,False,False,True,False,True,False
4456,4457.0,2.0,409.63,13.431575,False,False,False,False,False,False,True,False,True,False
4457,4458.0,1.0,941.84,79.937362,False,False,False,False,False,False,True,True,True,True
4458,4459.0,2.0,1451.93,31.273255,False,False,False,False,False,False,True,False,True,False
4459,4460.0,1.0,1827.07,2.823008,False,False,False,False,False,False,True,False,True,False


In [70]:
# export data to csv

df_did_clean.to_csv('LCFS_DiD_data.csv', encoding='utf-8', index=True)

In [68]:
df_did_filtered = df_did_clean[df_did_clean['p425'] == 1].copy()

In [69]:
df_did_filtered.to_csv('LCFS_DiD_data_filtered.csv', encoding='utf-8', index=True)